#### Choose a plain image P and a secret key K of 32 bytes.

##### Encryption

In [6]:
import numpy as np
from PIL import Image

def generate_latin_square(size):
    latin_square = np.arange(size)
    np.random.shuffle(latin_square)
    return latin_square

# Step 2: Latin Square Whitening
def latin_square_whitening(data, latin_squares):
    # Get the dimensions of the image
    height, width, channels = data.shape

    # Apply Latin square whitening using XOR for each channel
    whitened_data = np.zeros_like(data, dtype=np.uint8)
    for channel in range(channels):
        whitened_data[:, :, channel] = data[:, :, channel] ^ latin_squares[channel][data[:, :, channel]]

    return whitened_data

# Step 3: Latin Square Substitution
def latin_square_substitution(data, latin_square, key):
    # Apply Latin square substitution
    substituted_data = np.array([latin_square[byte] for byte in data], dtype=np.uint8)
    
    # XOR with the encryption key for each element
    key = np.frombuffer(key, dtype=np.uint8)  # Convert the key to the same data type
    key = np.tile(key, len(substituted_data) // len(key) + 1)[:len(substituted_data)]  # Reshape the key
    substituted_data = substituted_data ^ key
    
    return substituted_data


# Step 4: Latin Square Permutation
def latin_square_permutation(data, latin_square, key):
    # Get the size of the data
    size = len(data)
    
    # Apply Latin square permutation
    permuted_data = np.zeros(size, dtype=np.uint8)
    for i, byte in enumerate(data):
        bit_position = latin_square[i]
        permuted_data[bit_position] = byte
    
    # XOR with the encryption key for each element
    key = np.frombuffer(key, dtype=np.uint8)  # Convert the key to the same data type
    key = np.tile(key, size // len(key) + 1)[:size]  # Reshape the key
    permuted_data = permuted_data ^ key
    
    return permuted_data



# Main program
if __name__ == "__main__":
    # Load an example image (replace with your image loading code)
    image_path = "Crew 6 launch .jpg"  # Replace with the path to your image file
    image = Image.open(image_path)

    # Convert the image to a numpy array
    image_data = np.array(image)

    # Step 1: Generate a secret key K (32 bytes)
    secret_key = np.random.bytes(32)

    # Generate Latin squares for each channel
    channels = image_data.shape[-1]
    latin_squares = [generate_latin_square(256) for _ in range(channels)]

    # Step 2: Latin Square Whitening
    whitened_data = latin_square_whitening(image_data, latin_squares)

    # Generate a Latin square (substitution matrix) for the given size
    latin_square = generate_latin_square(image_data.size)

    # Step 3: Latin Square Substitution
    substituted_data = latin_square_substitution(whitened_data.flatten(), latin_square, secret_key)

    # Step 4: Latin Square Permutation
    permuted_data = latin_square_permutation(substituted_data, latin_square, secret_key)

    # After completing all encryption rounds, you can obtain the encrypted image
    encrypted_image = permuted_data.reshape(image_data.shape)

    # Display the encrypted image
    encrypted_image = Image.fromarray(encrypted_image)
    encrypted_image.show()  # Show the image in a default image viewer

    # Alternatively, you can save the encrypted image to a file
    encrypted_image.save("encrypted_image.jpg")


##### Decryption

In [7]:
import numpy as np
from PIL import Image

# Define Latin square data (replace with your actual Latin squares)
# latin_squares = [...]  # Fill in your Latin square data
# Generate Latin squares for each channel
channels = image_data.shape[-1]
latin_squares = [generate_latin_square(256) for _ in range(channels)]

# Inverse Latin Square Permutation
def latin_square_inverse_permutation(data, latin_square, key):
    size = len(data)
    permuted_data = np.zeros(size, dtype=np.uint8)
    for i, byte in enumerate(data):
        bit_position = np.where(latin_square == i)[0][0]
        permuted_data[bit_position] = byte

    key = np.frombuffer(key, dtype=np.uint8)
    key = np.tile(key, size // len(key) + 1)[:size]
    permuted_data = permuted_data ^ key

    return permuted_data

# Inverse Latin Square Substitution
def latin_square_inverse_substitution(data, latin_square, key):
    key = np.frombuffer(key, dtype=np.uint8)
    key = np.tile(key, len(data) // len(key) + 1)[:len(data)]
    substituted_data = data ^ key

    inverse_latin_square = np.argsort(latin_square)
    inverted_data = np.array([inverse_latin_square[byte] for byte in substituted_data], dtype=np.uint8)

    return inverted_data

# Inverse Latin Square Whitening
def latin_square_inverse_whitening(data, key, latin_squares):
    height, width, channels = data.shape
    inverse_whitened_data = np.zeros_like(data, dtype=np.uint8)
    for channel in range(channels):
        inverse_whitened_data[:, :, channel] = data[:, :, channel] ^ latin_squares[channel][data[:, :, channel]]

    return inverse_whitened_data

if __name__ == "__main":
    encrypted_image_path = "encrypted_image.jpg"  # Replace with the path to the encrypted image file
    encrypted_image = Image.open(encrypted_image_path)
    encrypted_image_data = np.array(encrypted_image)

    secret_key = np.random.bytes(32)

    # Step 1: Inverse Latin Square Whitening
    whitened_data = latin_square_inverse_whitening(encrypted_image_data, secret_key, latin_squares)

    # Step 2: Inverse Latin Square Substitution
    substituted_data = latin_square_inverse_substitution(whitened_data.flatten(), latin_squares[0], secret_key)

    # Step 3: Inverse Latin Square Permutation
    permuted_data = latin_square_inverse_permutation(substituted_data, latin_squares[0], secret_key)

    decrypted_image_data = permuted_data.reshape(encrypted_image_data.shape)

    decrypted_image = Image.fromarray(decrypted_image_data)
    decrypted_image.show()  # Show the decrypted image
    decrypted_image.save("decrypted_image.jpg")  # Save the decrypted image to a file


#### Decryption

In [12]:
import numpy as np
from PIL import Image

# Define your actual Latin squares used in encryption here
# Define your actual Latin squares used in encryption here
channels = image_data.shape[-1]
latin_squares = [generate_latin_square(256) for _ in range(channels)]


# Inverse Latin Square Permutation
def latin_square_inverse_permutation(data, latin_square, key):
    size = len(data)
    permuted_data = np.zeros(size, dtype=np.uint8)
    for i, byte in enumerate(data):
        bit_position = np.where(latin_square == i)[0][0]
        permuted_data[bit_position] = byte

    key = np.frombuffer(key, dtype=np.uint8)
    key = np.tile(key, size // len(key) + 1)[:size]
    permuted_data = permuted_data ^ key

    return permuted_data

# Inverse Latin Square Substitution
def latin_square_inverse_substitution(data, latin_square, key):
    key = np.frombuffer(key, dtype=np.uint8)
    key = np.tile(key, len(data) // len(key) + 1)[:len(data)]
    substituted_data = data ^ key

    inverse_latin_square = np.argsort(latin_square)
    inverted_data = np.array([inverse_latin_square[byte] for byte in substituted_data], dtype=np.uint8)

    return inverted_data

# Inverse Latin Square Whitening
def latin_square_inverse_whitening(data, key, latin_squares):
    height, width, channels = data.shape
    inverse_whitened_data = np.zeros_like(data, dtype=np.uint8)
    for channel in range(channels):
        inverse_whitened_data[:, :, channel] = data[:, :, channel] ^ latin_squares[channel][data[:, :, channel]]

    return inverse_whitened_data

if __name__ == "__main__":
    encrypted_image_path = "encrypted_image.jpg"  # Replace with the path to the encrypted image file
    encrypted_image = Image.open(encrypted_image_path)
    encrypted_image_data = np.array(encrypted_image)

    secret_key = np.random.bytes(32)

    # Step 1: Inverse Latin Square Whitening
    whitened_data = latin_square_inverse_whitening(encrypted_image_data, secret_key, latin_squares)

    # Step 2: Inverse Latin Square Substitution
    substituted_data = latin_square_inverse_substitution(whitened_data.flatten(), latin_squares[0], secret_key)

    # Step 3: Inverse Latin Square Permutation
    permuted_data = latin_square_inverse_permutation(substituted_data, latin_squares[0], secret_key)

    decrypted_image_data = permuted_data.reshape(encrypted_image_data.shape)

    decrypted_image = Image.fromarray(decrypted_image_data)
    decrypted_image.show()  # Show the decrypted image
    decrypted_image.save("decrypted_image.jpg")  # Save the decrypted image to a file


IndexError: index 0 is out of bounds for axis 0 with size 0